#**Fuzzy**

###Fuzzy Support Vector Machine (Type Fuzzy SVM)

In [ ]:
#GPT의 fuzzy function
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_m_loss(xi, c): # c는 퍼지 함수의 스무딩 파라미터
    """ Fuzzy membership function for slack variables. """
    return (xi**2) / (c + xi**2)

def objective(params, X, y, C, c): # C는 정규화 강도를 조절하는 하이퍼파라미터
    n = X.shape[1]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    # Regularization term
    regularization = 0.5 * np.dot(w, w)
    # Non-linear cost for slack variables using a fuzzy membership function
    slack_term = C * np.sum(fuzzy_m_loss(xi, c))
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    # Linear constraints: y(w·x + b) >= 1 - ξ_i
    linear_constraints = y * (X.dot(w) + b) - 1 + xi
    return linear_constraints

def solve_svm(X, y, C=1.0, c=1.0):
    m, n = X.shape
    # Initial parameter settings
    initial_params = np.zeros(n + 1 + m)  # w, b, ξ initial values
    # Constraint settings
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0
    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, c),
        constraints=cons,
        method='SLSQP'
    )
    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

# Execution
w, b = solve_svm(X, y, C=1.0, c=1.0)

# Prediction and accuracy calculation
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

# Output results
print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


NameError: name 'X' is not defined

In [ ]:
#논문1의 fuzzy function
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_m_loss(xi, y, sigma_squared):
    """ Fuzzy membership function for slack variables with class-based differentiation. """
    norm_xi_squared = np.linalg.norm(xi)**2
    if y == 1:
        return 1 - np.exp(-norm_xi_squared / sigma_squared)
    else:
        return np.exp(-norm_xi_squared / sigma_squared)

def objective(params, X, y, C, sigma_squared):
    n = X.shape[1]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    # Regularization term
    regularization = 0.5 * np.dot(w, w)
    # Non-linear cost for slack variables using class-based fuzzy membership function
    slack_term = C * np.sum([fuzzy_m_loss(xi[i], y[i], sigma_squared) for i in range(len(y))])
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    # Linear constraints: y(w·x + b) >= 1 - ξ_i
    return y * (X.dot(w) + b) - 1 + xi

def solve_svm(X, y, C=1.0, sigma_squared=1.0):
    m, n = X.shape
    # Initial parameter settings
    initial_params = np.zeros(n + 1 + m)  # w, b, ξ initial values
    # Constraint settings
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0
    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, sigma_squared),
        constraints=cons,
        method='SLSQP'
    )
    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

w, b = solve_svm(X, y, C=1.0, sigma_squared=2.0)

# Prediction and accuracy calculation
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

# Output results
print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.61515606e-06 -5.08646116e-07]
Bias (b): 1.277453601099316
Accuracy: 50.00%


### 얘가 우리랑 비슷

In [ ]:
#논문 2,3의 fuzzy function
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, delta=0.1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 - np.linalg.norm(x - mean) / (r + delta)
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 - np.linalg.norm(x - mean) / (r + delta)

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, delta=0.1):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, delta) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0, delta=0.1)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.3141682  -0.13361577]
Bias (b): 0.03292133355183146
Accuracy: 87.00%


###Variation of the Third Fuzzy Function

Version of using delta

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, delta=0.1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / ( 1 + np.linalg.norm(x - mean) / (r + delta))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / ( 1 + np.linalg.norm(x - mean) / (r + delta))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, delta=0.1):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, delta) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0, delta=0.1)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Weight vector (w): [ 1.32034644 -0.21151527]
Bias (b): 0.053031743210899165
Accuracy: 85.50%


1) Normalized distance reciprocal membership function

In [ ]:
# alpha = 1
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 1):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Weight vector (w): [ 1.32034644 -0.21151527]
Bias (b): 0.05303174312403852
Accuracy: 85.50%


2) Adjusted Normalized distance reciprocal membership function 1

In [ ]:
# alpha = 1.2

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 1.2):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Weight vector (w): [ 1.32034644 -0.21151527]
Bias (b): 0.0530317430693789
Accuracy: 85.50%


In [ ]:
# alpha = 1.5

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 1.5):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Weight vector (w): [ 1.29195985 -0.1944083 ]
Bias (b): 0.039857628291188635
Accuracy: 87.00%


In [ ]:
# alpha = 1.7

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 1.7):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.28809918 -0.18854413]
Bias (b): 0.04073196139390204
Accuracy: 87.50%


In [ ]:
# alpha = 2

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha * (1 + np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 2):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")

#alpha 값이 2일 때 "경계 데이터들의 영향력이 가장 애매한 상태"가 되기 때문에 정확도가 하락하는 것!
#alpha=3, 4, 5에서는 결정 경계(SVM 분류면)가 거의 같기 때문에 정확도도 동일할 가능성이 큼.
#Alpha=1일 때는 fuzzy membership이 기존보다 훨씬 더 커질 가능성이 높음.

Weight vector (w): [ 1.26302903 -0.1830263 ]
Bias (b): 0.05513454007221643
Accuracy: 86.50%


3) Adjusted Normalized distance reciprocal membership function 2

In [ ]:
# alpha = 2

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha + ( np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha + ( np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 2):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.28504593 -0.21850594]
Bias (b): 0.054487196653086586
Accuracy: 85.50%


In [ ]:
# alpha = 3

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha + ( np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha + ( np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 3):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.2319358  -0.20109982]
Bias (b): 0.03134837205745591
Accuracy: 86.00%


In [ ]:
# alpha = 4

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha + ( np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha + ( np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 4):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.2319358  -0.20109982]
Bias (b): 0.031348372053523206
Accuracy: 86.00%


In [ ]:
# alpha = 5

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (alpha + ( np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (alpha + ( np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 5):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.21027157 -0.18374945]
Bias (b): 0.04062271387586212
Accuracy: 86.00%


4) Adjusted Normalized distance reciprocal membership function 3

In [ ]:
# alpha = 2

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (1 + alpha * (np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (1 + alpha * (np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 2):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.32457413 -0.18662626]
Bias (b): 0.010828514810065318
Accuracy: 87.50%


In [ ]:
# alpha = 3

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (1 + alpha * (np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (1 + alpha * (np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 3):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.34534604 -0.18848138]
Bias (b): 0.027304650544193995
Accuracy: 87.50%


In [ ]:
# alpha = 4

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (1 + alpha * (np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (1 + alpha * (np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 4):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.3521967  -0.18920996]
Bias (b): 0.02228490901089509
Accuracy: 87.50%


In [ ]:
# alpha = 5

import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

def fuzzy_membership(x, y, class_mean, class_radius, alpha = 1):
    """
    Computes the fuzzy membership for a data point x based on its class.
    """
    if y == 1:
        r = class_radius[1]
        mean = class_mean[1]
        return 1 / (1 + alpha * (np.linalg.norm(x - mean) / r))
    else:
        r = class_radius[-1]
        mean = class_mean[-1]
        return 1 / (1 + alpha * (np.linalg.norm(x - mean) / r))

def objective(params, X, y, C, fuzzy_weights):
    """
    Objective function for the Fuzzy SVM.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]

    # Regularization term
    regularization = 0.5 * np.dot(w, w)

    # Weighted slack variables term
    slack_term = C * np.sum(fuzzy_weights * xi)

    return regularization + slack_term

def constraints(params, X, y):
    """
    Constraints for the Fuzzy SVM optimization.
    """
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha = 5):
    """
    Solves the Fuzzy SVM optimization problem.
    """
    m, n = X.shape

    # Calculate class means and radii
    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }
    class_radii = {
        1: np.max([np.linalg.norm(x - class_means[1]) for x in X[y == 1]]),
        -1: np.max([np.linalg.norm(x - class_means[-1]) for x in X[y == -1]])
    }

    # Compute fuzzy membership weights
    fuzzy_weights = np.array([fuzzy_membership(X[i], y[i], class_means, class_radii, alpha) for i in range(m)])

    # Initial parameters (w, b, xi)
    initial_params = np.zeros(n + 1 + m)

    # Constraints
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # ξ_i >= 0

    # Optimization
    result = minimize(
        fun=objective,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights),
        constraints=cons,
        method='SLSQP'
    )

    # Extract parameters
    params = result.x
    w = params[:n]
    b = params[n]

    return w, b

w, b = solve_fuzzy_svm(X, y, C=1.0)

# Prediction
y_pred = np.sign(np.dot(X, w) + b)
accuracy = accuracy_score(y, y_pred)

print(f"Weight vector (w): {w}")
print(f"Bias (b): {b}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Weight vector (w): [ 1.3521967  -0.18920996]
Bias (b): 0.02228490891873141
Accuracy: 87.50%


###NEW

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

## Complementary Logistic Fuzzy Membership Function
def fuzzy_membership(x, y, class_mean, alpha=1.0, beta=1.0):
  center = class_mean[y]
  distance = np.linalg.norm(x - center)
  membership = 1 - 1 / (1 + np.exp(-alpha * (distance - beta)))
  return membership

def objective(params, X, y, C, fuzzy_weights):
  n = X.shape[1]
  m = X.shape[0]
  w = params[:n]
  b = params[n]
  xi = params[n+1:]
  regularization = 0.5 * np.dot(w, w)
  slack_term = C * np.sum(fuzzy_weights * xi)
  return regularization + slack_term

def constraints(params, X, y):
  n = X.shape[1]
  m = X.shape[0]
  w = params[:n]
  b = params[n]
  xi = params[n+1:]
  return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_svm(X, y, C=1.0, alpha=1.0, beta=1.0):
  m, n = X.shape

  # Compute class means
  class_means = {
      1: np.mean(X[y == 1], axis=0),
      -1: np.mean(X[y == -1], axis=0)
  }

  # Compute fuzzy weights
  fuzzy_weights = np.array([
      fuzzy_membership(X[i], y[i], class_means, alpha, beta) for i in range(m)
  ])

  # Initialize parameters: [w(n), b(1), xi(m)]
  initial_params = np.zeros(n+1+m)

  # Constraints
  cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
  cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]  # xi >= 0

  # Solve optimization problem
  result = minimize(
      fun = objective,
      x0 = initial_params,
      args = (X, y, C, fuzzy_weights),
      constraints = cons,
      method = 'SLSQP'
  )

  # Extract w and b
  params = result.x
  w = params[:n]
  b = params[n]

  return w, b

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0.05, class_sep=1.5, random_state=42)
y = 2 * y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 훈련 및 평가
w, b = solve_fuzzy_svm(X_train, y_train, C=1.0, alpha=1.0, beta=1.0)
y_train_pred = np.sign(np.dot(X_train, w) + b)
y_test_pred = np.sign(np.dot(X_test, w) + b)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_accuracy, test_accuracy

(0.9214285714285714, 0.9166666666666666)

## M-estimator 기반 fuzzy SVM

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

## Complementary Logistic Fuzzy Membership Function
def fuzzy_membership(x, y, class_mean, alpha=1.0, beta=1.0):
    center = class_mean[y]
    distance = np.linalg.norm(x - center)
    return 1 - 1 / (1 + np.exp(-alpha * (distance - beta)))

def fair_loss(xi, c):
    return c**2 * ((xi / c) - np.log(1 + (xi / c)))

def cauchy_loss(xi, c):
    return 0.5 * c**2 * np.log(1 + (xi / c)**2)

def welsch_loss(xi, c):
    return 0.5 * c**2 * (1 - np.exp(-(xi / c)**2))

def gm_loss(xi, c):
    return 0.5 * (xi**2 / (1 + xi**2))

def objective_fuzzy_m(params, X, y, C, fuzzy_weights, loss_type, c):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    regularization = 0.5 * np.dot(w, w)

    if loss_type == 'fair':
        losses = fair_loss(xi, c)
    elif loss_type == 'cauchy':
        losses = cauchy_loss(xi, c)
    elif loss_type == 'welsch':
        losses = welsch_loss(xi, c)
    elif loss_type == 'geman_mcclure':
        losses = gm_loss(xi, c)
    else:
        raise ValueError("Unknown loss type")

    slack_term = C * np.sum(fuzzy_weights * losses)
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_mestimator_svm(X, y, C=1.0, alpha=5.0, beta=1.0, c=1.0, loss_type='fair'):
    m, n = X.shape

    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }

    fuzzy_weights = np.array([
        fuzzy_membership(X[i], y[i], class_means, alpha, beta) for i in range(m)
    ])

    initial_params = np.zeros(n + 1 + m)
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]

    result = minimize(
        fun=objective_fuzzy_m,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights, loss_type, c),
        constraints=cons,
        method='SLSQP'
    )

    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0.05, class_sep=1.5, random_state=42)
y = 2 * y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 재훈련 및 테스트
w, b = solve_fuzzy_mestimator_svm(X_train, y_train, C=1.0, alpha=5.0, beta=1.0, c=1.0, loss_type='fair')

# 예측 및 정확도 평가
y_train_pred = np.sign(np.dot(X_train, w) + b)
y_test_pred = np.sign(np.dot(X_test, w) + b)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_accuracy, test_accuracy

(0.9142857142857143, 0.9166666666666666)

## parameter 해석
- alpha: 감쇠 속도 (decay rate)
  - sigmoid의 기울기 결정
  - 클수록 중심 근처에서 급격하게 membership이 감소
  - 작을수록 완만한 감소 -> 모든 샘플에 대해 상대적으로 비슷한 weight

- beta: 중심 기준 거리 (threshold distance)
  - sigmoid의 x축 이동: 일정 거리까지는 거의 1로 보고, 그 이후 급감을 조절
  - 클래스 중심에서 일정 거리떨어진 지점에서 mambership이 ~~ 0.5

### alpha = 0.5, beta = 0.25

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

## Complementary Logistic Fuzzy Membership Function
def fuzzy_membership(x, y, class_mean, alpha=0.5, beta=0.25):
    center = class_mean[y]
    distance = np.linalg.norm(x - center)
    return 1 - 1 / (1 + np.exp(-alpha * (distance - beta)))

def fair_loss(xi, c):
    return c**2 * ((xi / c) - np.log(1 + (xi / c)))

def cauchy_loss(xi, c):
    return 0.5 * c**2 * np.log(1 + (xi / c)**2)

def welsch_loss(xi, c):
    return 0.5 * c**2 * (1 - np.exp(-(xi / c)**2))

def gm_loss(xi, c):
    return 0.5 * (xi**2 / (1 + xi**2))

def objective_fuzzy_m(params, X, y, C, fuzzy_weights, loss_type, c):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    regularization = 0.5 * np.dot(w, w)

    if loss_type == 'fair':
        losses = fair_loss(xi, c)
    elif loss_type == 'cauchy':
        losses = cauchy_loss(xi, c)
    elif loss_type == 'welsch':
        losses = welsch_loss(xi, c)
    elif loss_type == 'geman_mcclure':
        losses = gm_loss(xi, c)
    else:
        raise ValueError("Unknown loss type")

    slack_term = C * np.sum(fuzzy_weights * losses)
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_mestimator_svm(X, y, C=1.0, alpha=0.5, beta=0.25, c=1.0, loss_type='geman_mcclure'):
    m, n = X.shape

    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }

    fuzzy_weights = np.array([
        fuzzy_membership(X[i], y[i], class_means, alpha, beta) for i in range(m)
    ])

    initial_params = np.zeros(n + 1 + m)
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]

    result = minimize(
        fun=objective_fuzzy_m,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights, loss_type, c),
        constraints=cons,
        method='SLSQP'
    )

    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0.05, class_sep=1.5, random_state=42)
y = 2 * y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 재훈련 및 테스트
w, b = solve_fuzzy_mestimator_svm(X_train, y_train, C=1.0, alpha=0.5, beta=0.25, c=1.0, loss_type='geman_mcclure')

# 예측 및 정확도 평가
y_train_pred = np.sign(np.dot(X_train, w) + b)
y_test_pred = np.sign(np.dot(X_test, w) + b)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_accuracy, test_accuracy

(0.9142857142857143, 0.9166666666666666)

### alpha = 0.5, beta = 0.5

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

## Complementary Logistic Fuzzy Membership Function
def fuzzy_membership(x, y, class_mean, alpha=0.5, beta=0.5):
    center = class_mean[y]
    distance = np.linalg.norm(x - center)
    return 1 - 1 / (1 + np.exp(-alpha * (distance - beta)))

def fair_loss(xi, c):
    return c**2 * ((xi / c) - np.log(1 + (xi / c)))

def cauchy_loss(xi, c):
    return 0.5 * c**2 * np.log(1 + (xi / c)**2)

def welsch_loss(xi, c):
    return 0.5 * c**2 * (1 - np.exp(-(xi / c)**2))

def gm_loss(xi, c):
    return 0.5 * (xi**2 / (1 + xi**2))

def objective_fuzzy_m(params, X, y, C, fuzzy_weights, loss_type, c):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    regularization = 0.5 * np.dot(w, w)

    if loss_type == 'fair':
        losses = fair_loss(xi, c)
    elif loss_type == 'cauchy':
        losses = cauchy_loss(xi, c)
    elif loss_type == 'welsch':
        losses = welsch_loss(xi, c)
    elif loss_type == 'geman_mcclure':
        losses = gm_loss(xi, c)
    else:
        raise ValueError("Unknown loss type")

    slack_term = C * np.sum(fuzzy_weights * losses)
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_mestimator_svm(X, y, C=1.0, alpha=0.5, beta=0.5, c=1.0, loss_type='geman_mcclure'):
    m, n = X.shape

    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }

    fuzzy_weights = np.array([
        fuzzy_membership(X[i], y[i], class_means, alpha, beta) for i in range(m)
    ])

    initial_params = np.zeros(n + 1 + m)
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]

    result = minimize(
        fun=objective_fuzzy_m,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights, loss_type, c),
        constraints=cons,
        method='SLSQP'
    )

    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0.05, class_sep=1.5, random_state=42)
y = 2 * y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 재훈련 및 테스트
w, b = solve_fuzzy_mestimator_svm(X_train, y_train, C=1.0, alpha=0.5, beta=0.5, c=1.0, loss_type='geman_mcclure')

# 예측 및 정확도 평가
y_train_pred = np.sign(np.dot(X_train, w) + b)
y_test_pred = np.sign(np.dot(X_test, w) + b)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_accuracy, test_accuracy

(0.9142857142857143, 0.9166666666666666)

### alpha = 0.5, beta = 1.0

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

## Complementary Logistic Fuzzy Membership Function
def fuzzy_membership(x, y, class_mean, alpha=0.5, beta=1.0):
    center = class_mean[y]
    distance = np.linalg.norm(x - center)
    return 1 - 1 / (1 + np.exp(-alpha * (distance - beta)))

def fair_loss(xi, c):
    return c**2 * ((xi / c) - np.log(1 + (xi / c)))

def cauchy_loss(xi, c):
    return 0.5 * c**2 * np.log(1 + (xi / c)**2)

def welsch_loss(xi, c):
    return 0.5 * c**2 * (1 - np.exp(-(xi / c)**2))

def gm_loss(xi, c):
    return 0.5 * (xi**2 / (1 + xi**2))

def objective_fuzzy_m(params, X, y, C, fuzzy_weights, loss_type, c):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    regularization = 0.5 * np.dot(w, w)

    if loss_type == 'fair':
        losses = fair_loss(xi, c)
    elif loss_type == 'cauchy':
        losses = cauchy_loss(xi, c)
    elif loss_type == 'welsch':
        losses = welsch_loss(xi, c)
    elif loss_type == 'geman_mcclure':
        losses = gm_loss(xi, c)
    else:
        raise ValueError("Unknown loss type")

    slack_term = C * np.sum(fuzzy_weights * losses)
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_mestimator_svm(X, y, C=1.0, alpha=0.5, beta=1.0, c=1.0, loss_type='geman_mcclure'):
    m, n = X.shape

    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }

    fuzzy_weights = np.array([
        fuzzy_membership(X[i], y[i], class_means, alpha, beta) for i in range(m)
    ])

    initial_params = np.zeros(n + 1 + m)
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]

    result = minimize(
        fun=objective_fuzzy_m,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights, loss_type, c),
        constraints=cons,
        method='SLSQP'
    )

    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0.05, class_sep=1.5, random_state=42)
y = 2 * y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 재훈련 및 테스트
w, b = solve_fuzzy_mestimator_svm(X_train, y_train, C=1.0, alpha=0.5, beta=1.0, c=1.0, loss_type='geman_mcclure')

# 예측 및 정확도 평가
y_train_pred = np.sign(np.dot(X_train, w) + b)
y_test_pred = np.sign(np.dot(X_test, w) + b)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_accuracy, test_accuracy

(0.9142857142857143, 0.9166666666666666)

### alpha = 1.0, beta = 0.25

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

## Complementary Logistic Fuzzy Membership Function
def fuzzy_membership(x, y, class_mean, alpha=1.0, beta=0.25):
    center = class_mean[y]
    distance = np.linalg.norm(x - center)
    return 1 - 1 / (1 + np.exp(-alpha * (distance - beta)))

def fair_loss(xi, c):
    return c**2 * ((xi / c) - np.log(1 + (xi / c)))

def cauchy_loss(xi, c):
    return 0.5 * c**2 * np.log(1 + (xi / c)**2)

def welsch_loss(xi, c):
    return 0.5 * c**2 * (1 - np.exp(-(xi / c)**2))

def gm_loss(xi, c):
    return 0.5 * (xi**2 / (1 + xi**2))

def objective_fuzzy_m(params, X, y, C, fuzzy_weights, loss_type, c):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    regularization = 0.5 * np.dot(w, w)

    if loss_type == 'fair':
        losses = fair_loss(xi, c)
    elif loss_type == 'cauchy':
        losses = cauchy_loss(xi, c)
    elif loss_type == 'welsch':
        losses = welsch_loss(xi, c)
    elif loss_type == 'geman_mcclure':
        losses = gm_loss(xi, c)
    else:
        raise ValueError("Unknown loss type")

    slack_term = C * np.sum(fuzzy_weights * losses)
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_mestimator_svm(X, y, C=1.0, alpha=1.0, beta=0.25, c=1.0, loss_type='geman_mcclure'):
    m, n = X.shape

    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }

    fuzzy_weights = np.array([
        fuzzy_membership(X[i], y[i], class_means, alpha, beta) for i in range(m)
    ])

    initial_params = np.zeros(n + 1 + m)
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]

    result = minimize(
        fun=objective_fuzzy_m,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights, loss_type, c),
        constraints=cons,
        method='SLSQP'
    )

    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0.05, class_sep=1.5, random_state=42)
y = 2 * y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 재훈련 및 테스트
w, b = solve_fuzzy_mestimator_svm(X_train, y_train, C=1.0, alpha=1.0, beta=0.25, c=1.0, loss_type='geman_mcclure')

# 예측 및 정확도 평가
y_train_pred = np.sign(np.dot(X_train, w) + b)
y_test_pred = np.sign(np.dot(X_test, w) + b)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_accuracy, test_accuracy

(0.9214285714285714, 0.9166666666666666)

### alpha = 1.0, beta = 0.5

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

## Complementary Logistic Fuzzy Membership Function
def fuzzy_membership(x, y, class_mean, alpha=1.0, beta=0.5):
    center = class_mean[y]
    distance = np.linalg.norm(x - center)
    return 1 - 1 / (1 + np.exp(-alpha * (distance - beta)))

def fair_loss(xi, c):
    return c**2 * ((xi / c) - np.log(1 + (xi / c)))

def cauchy_loss(xi, c):
    return 0.5 * c**2 * np.log(1 + (xi / c)**2)

def welsch_loss(xi, c):
    return 0.5 * c**2 * (1 - np.exp(-(xi / c)**2))

def gm_loss(xi, c):
    return 0.5 * (xi**2 / (1 + xi**2))

def objective_fuzzy_m(params, X, y, C, fuzzy_weights, loss_type, c):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    regularization = 0.5 * np.dot(w, w)

    if loss_type == 'fair':
        losses = fair_loss(xi, c)
    elif loss_type == 'cauchy':
        losses = cauchy_loss(xi, c)
    elif loss_type == 'welsch':
        losses = welsch_loss(xi, c)
    elif loss_type == 'geman_mcclure':
        losses = gm_loss(xi, c)
    else:
        raise ValueError("Unknown loss type")

    slack_term = C * np.sum(fuzzy_weights * losses)
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_mestimator_svm(X, y, C=1.0, alpha=1.0, beta=0.5, c=1.0, loss_type='geman_mcclure'):
    m, n = X.shape

    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }

    fuzzy_weights = np.array([
        fuzzy_membership(X[i], y[i], class_means, alpha, beta) for i in range(m)
    ])

    initial_params = np.zeros(n + 1 + m)
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]

    result = minimize(
        fun=objective_fuzzy_m,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights, loss_type, c),
        constraints=cons,
        method='SLSQP'
    )

    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0.05, class_sep=1.5, random_state=42)
y = 2 * y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 재훈련 및 테스트
w, b = solve_fuzzy_mestimator_svm(X_train, y_train, C=1.0, alpha=1.0, beta=0.5, c=1.0, loss_type='geman_mcclure')

# 예측 및 정확도 평가
y_train_pred = np.sign(np.dot(X_train, w) + b)
y_test_pred = np.sign(np.dot(X_test, w) + b)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_accuracy, test_accuracy

(0.9214285714285714, 0.9166666666666666)

### alpha = 1.0, beta = 1.0

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score

## Complementary Logistic Fuzzy Membership Function
def fuzzy_membership(x, y, class_mean, alpha=1.0, beta=1.0):
    center = class_mean[y]
    distance = np.linalg.norm(x - center)
    return 1 - 1 / (1 + np.exp(-alpha * (distance - beta)))

def fair_loss(xi, c):
    return c**2 * ((xi / c) - np.log(1 + (xi / c)))

def cauchy_loss(xi, c):
    return 0.5 * c**2 * np.log(1 + (xi / c)**2)

def welsch_loss(xi, c):
    return 0.5 * c**2 * (1 - np.exp(-(xi / c)**2))

def gm_loss(xi, c):
    return 0.5 * (xi**2 / (1 + xi**2))

def objective_fuzzy_m(params, X, y, C, fuzzy_weights, loss_type, c):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    regularization = 0.5 * np.dot(w, w)

    if loss_type == 'fair':
        losses = fair_loss(xi, c)
    elif loss_type == 'cauchy':
        losses = cauchy_loss(xi, c)
    elif loss_type == 'welsch':
        losses = welsch_loss(xi, c)
    elif loss_type == 'geman_mcclure':
        losses = gm_loss(xi, c)
    else:
        raise ValueError("Unknown loss type")

    slack_term = C * np.sum(fuzzy_weights * losses)
    return regularization + slack_term

def constraints(params, X, y):
    n = X.shape[1]
    m = X.shape[0]
    w = params[:n]
    b = params[n]
    xi = params[n+1:]
    return y * (X.dot(w) + b) - 1 + xi

def solve_fuzzy_mestimator_svm(X, y, C=1.0, alpha=1.0, beta=1.0, c=1.0, loss_type='geman_mcclure'):
    m, n = X.shape

    class_means = {
        1: np.mean(X[y == 1], axis=0),
        -1: np.mean(X[y == -1], axis=0)
    }

    fuzzy_weights = np.array([
        fuzzy_membership(X[i], y[i], class_means, alpha, beta) for i in range(m)
    ])

    initial_params = np.zeros(n + 1 + m)
    cons = [{'type': 'ineq', 'fun': lambda params, i=i: constraints(params, X, y)[i]} for i in range(m)]
    cons += [{'type': 'ineq', 'fun': lambda params, i=i: params[n + 1 + i]} for i in range(m)]

    result = minimize(
        fun=objective_fuzzy_m,
        x0=initial_params,
        args=(X, y, C, fuzzy_weights, loss_type, c),
        constraints=cons,
        method='SLSQP'
    )

    params = result.x
    w = params[:n]
    b = params[n]
    return w, b

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1,
                           flip_y=0.05, class_sep=1.5, random_state=42)
y = 2 * y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 재훈련 및 테스트
w, b = solve_fuzzy_mestimator_svm(X_train, y_train, C=1.0, alpha=1.0, beta=1.0, c=1.0, loss_type='geman_mcclure')

# 예측 및 정확도 평가
y_train_pred = np.sign(np.dot(X_train, w) + b)
y_test_pred = np.sign(np.dot(X_test, w) + b)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_accuracy, test_accuracy

(0.9214285714285714, 0.9166666666666666)

### M-estimator, parameter에 따른 변화 (alpha: 0.5, 1.0 / beta: 0.25, 0.5, 1.0)
- Fair
  - alpha=0.5, beta : 0.9333
  - alpha=1.0, beta : 0.9166
- Cauchy
  - alpha=0.5, beta : 0.9333
  - alpha=1.0, beta : 0.9166
- Welsch
  - alpha=0.5, beta : 0.9166
  - alpha=1.0, beta : 0.9166
- G.M.
  - alpha=0.5, beta : 0.9166
  - alpha=1.0, beta : 0.9166